---
title: "Multinomial Logit Model"
author: "Timon Ortwein"
date: today
---



This assignment expores two methods for estimating the MNL model: (1) via Maximum Likelihood, and (2) via a Bayesian approach using a Metropolis-Hastings MCMC algorithm. 


## 1. Likelihood for the Multi-nomial Logit (MNL) Model

Suppose we have $i=1,\ldots,n$ consumers who each select exactly one product $j$ from a set of $J$ products. The outcome variable is the identity of the product chosen $y_i \in \{1, \ldots, J\}$ or equivalently a vector of $J-1$ zeros and $1$ one, where the $1$ indicates the selected product. For example, if the third product was chosen out of 3 products, then either $y=3$ or $y=(0,0,1)$ depending on how we want to represent it. Suppose also that we have a vector of data on each product $x_j$ (eg, brand, price, etc.). 

We model the consumer's decision as the selection of the product that provides the most utility, and we'll specify the utility function as a linear function of the product characteristics:

$$ U_{ij} = x_j'\beta + \epsilon_{ij} $$

where $\epsilon_{ij}$ is an i.i.d. extreme value error term. 

The choice of the i.i.d. extreme value error term leads to a closed-form expression for the probability that consumer $i$ chooses product $j$:

$$ \mathbb{P}_i(j) = \frac{e^{x_j'\beta}}{\sum_{k=1}^Je^{x_k'\beta}} $$

For example, if there are 3 products, the probability that consumer $i$ chooses product 3 is:

$$ \mathbb{P}_i(3) = \frac{e^{x_3'\beta}}{e^{x_1'\beta} + e^{x_2'\beta} + e^{x_3'\beta}} $$

A clever way to write the individual likelihood function for consumer $i$ is the product of the $J$ probabilities, each raised to the power of an indicator variable ($\delta_{ij}$) that indicates the chosen product:

$$ L_i(\beta) = \prod_{j=1}^J \mathbb{P}_i(j)^{\delta_{ij}} = \mathbb{P}_i(1)^{\delta_{i1}} \times \ldots \times \mathbb{P}_i(J)^{\delta_{iJ}}$$

Notice that if the consumer selected product $j=3$, then $\delta_{i3}=1$ while $\delta_{i1}=\delta_{i2}=0$ and the likelihood is:

$$ L_i(\beta) = \mathbb{P}_i(1)^0 \times \mathbb{P}_i(2)^0 \times \mathbb{P}_i(3)^1 = \mathbb{P}_i(3) = \frac{e^{x_3'\beta}}{\sum_{k=1}^3e^{x_k'\beta}} $$

The joint likelihood (across all consumers) is the product of the $n$ individual likelihoods:

$$ L_n(\beta) = \prod_{i=1}^n L_i(\beta) = \prod_{i=1}^n \prod_{j=1}^J \mathbb{P}_i(j)^{\delta_{ij}} $$

And the joint log-likelihood function is:

$$ \ell_n(\beta) = \sum_{i=1}^n \sum_{j=1}^J \delta_{ij} \log(\mathbb{P}_i(j)) $$


## 2. Simulate Conjoint Data

We will simulate data from a conjoint experiment about video content streaming services. We elect to simulate 100 respondents, each completing 10 choice tasks, where they choose from three alternatives per task. For simplicity, there is not a "no choice" option; each simulated respondent must select one of the 3 alternatives. 

Each alternative is a hypothetical streaming offer consistent of three attributes: (1) brand is either Netflix, Amazon Prime, or Hulu; (2) ads can either be part of the experience, or it can be ad-free, and (3) price per month ranges from \$4 to \$32 in increments of \$4.

The part-worths (ie, preference weights or beta parameters) for the attribute levels will be 1.0 for Netflix, 0.5 for Amazon Prime (with 0 for Hulu as the reference brand); -0.8 for included adverstisements (0 for ad-free); and -0.1*price so that utility to consumer $i$ for hypothethical streaming service $j$ is 

$$
u_{ij} = (1 \times Netflix_j) + (0.5 \times Prime_j) + (-0.8*Ads_j) - 0.1\times Price_j + \varepsilon_{ij}
$$

where the variables are binary indicators and $\varepsilon$ is Type 1 Extreme Value (ie, Gumble) distributed.

The following code provides the simulation of the conjoint data.

:::: {.callout-note collapse="true"}

In [ ]:
import numpy as np
import pandas as pd

# set seed for reproducibility
np.random.seed(123)

# define attributes
brand = ["N", "P", "H"]  # Netflix, Prime, Hulu
ad = ["Yes", "No"]
price = list(range(8, 33, 4))

# generate all possible profiles
profiles = pd.DataFrame([
    (b, a, p) for b in brand for a in ad for p in price
], columns=["brand", "ad", "price"])
m = len(profiles)

# assign part-worth utilities (true parameters)
b_util = {"N": 1.0, "P": 0.5, "H": 0}
a_util = {"Yes": -0.8, "No": 0.0}
def p_util(p): return -0.1 * p

# number of respondents, choice tasks, and alternatives per task
n_peeps = 100
n_tasks = 10
n_alts = 3

# function to simulate one respondent's data
def sim_one(id):
    datlist = []
    for t in range(1, n_tasks + 1):
        dat = profiles.sample(n=n_alts).copy()
        dat.insert(0, "task", t)
        dat.insert(0, "resp", id)
        dat["v"] = (
            dat["brand"].map(b_util) +
            dat["ad"].map(a_util) +
            dat["price"].apply(p_util)
        ).round(10)
        dat["e"] = -np.log(-np.log(np.random.uniform(size=n_alts)))
        dat["u"] = dat["v"] + dat["e"]
        dat["choice"] = (dat["u"] == dat["u"].max()).astype(int)
        datlist.append(dat)
    return pd.concat(datlist, ignore_index=True)

# simulate data for all respondents
conjoint_data = pd.concat([sim_one(i) for i in range(1, n_peeps + 1)], ignore_index=True)

# remove values unobservable to the researcher
conjoint_data = conjoint_data[["resp", "task", "brand", "ad", "price", "choice"]]

# clean up
for name in dir():
    if name != "conjoint_data":
        del globals()[name]

print(conjoint_data.head())

::::



## 3. Preparing the Data for Estimation

The "hard part" of the MNL likelihood function is organizing the data, as we need to keep track of 3 dimensions (consumer $i$, covariate $k$, and product $j$) instead of the typical 2 dimensions for cross-sectional regression models (consumer $i$ and covariate $k$). The fact that each task for each respondent has the same number of alternatives (3) helps.  In addition, we need to convert the categorical variables for brand and ads into binary variables.

_todo: reshape and prep the data_ DONE

In [ ]:
conjoint_data['brand_netflix'] = (conjoint_data['brand'] == 'N').astype(int)
conjoint_data['brand_prime'] = (conjoint_data['brand'] == 'P').astype(int)
conjoint_data['ads_yes'] = (conjoint_data['ad'] == 'Yes').astype(int)

X = conjoint_data[['brand_netflix', 'brand_prime', 'ads_yes', 'price']].values
y = conjoint_data['choice'].values

respondent = conjoint_data['resp'].values
task = conjoint_data['task'].values

print(conjoint_data.head())

## 4. Estimation via Maximum Likelihood

We implement the MNL model using maximum likelihood estimation. The log-likelihood function is optimized using the BFGS algorithm to find the parameter estimates and their standard errors.


In [ ]:
import numpy as np

def mnl(beta, X, y, ids):
    v = X @ beta
    
    unique_tasks = np.unique(ids)
    log_likelihood = 0.0
    for t in unique_tasks:
        idx = (ids == t)
        v_t = v[idx]
        y_t = y[idx]

        denom = np.log(np.sum(np.exp(v_t)))
        log_likelihood += np.sum(y_t * (v_t - denom))

    return -log_likelihood

task_ids = conjoint_data['resp'].astype(str) + "_" + conjoint_data['task'].astype(str)
task_ids = task_ids.values

Now we use scipy.optimize to find the MLEs for the Netflix, Prime, Ads and Price parameter.


In [ ]:
from scipy.optimize import minimize

# Minimize the log-likelihood
result = minimize(
    mnl,
    x0=np.zeros(4),
    args=(X, y, task_ids),
    method='BFGS'
)

hessian = result.hess_inv
error = np.sqrt(np.diag(hessian))
ci_lower = result.x - 1.96 * error
ci_upper = result.x + 1.96 * error

results_df = pd.DataFrame({
    'Parameter': ['Netflix', 'Prime', 'Ads', 'Price'],
    'MLE Estimate': result.x,
    'Std. Error': error,
    '95% CI Lower': ci_lower,
    '95% CI Upper': ci_upper
})

print("\nMaximum Likelihood Estimation Results:")
print(results_df.to_string(index=False))

## 5. Estimation via Bayesian Methods

We implement a Metropolis-Hastings MCMC sampler to estimate the posterior distribution of the parameters. The sampler uses:
- 11,000 total steps with 1,000 burn-in iterations
- N(0,5) priors for binary variable coefficients
- N(0,1) prior for the price coefficient
- Multivariate normal proposal distribution


In [ ]:
#| label: bayesian-estimation
#| code-fold: true
import numpy as np

def log_prior(beta):
    # beta: array of length 4
    # First 3: N(0, 5^2), Last: N(0, 1^2)
    lp = 0
    lp += -0.5 * ((beta[0] / 5) ** 2 + np.log(2 * np.pi * 25))
    lp += -0.5 * ((beta[1] / 5) ** 2 + np.log(2 * np.pi * 25))
    lp += -0.5 * ((beta[2] / 5) ** 2 + np.log(2 * np.pi * 25))
    lp += -0.5 * ((beta[3] / 1) ** 2 + np.log(2 * np.pi * 1))
    return lp

def log_posterior(beta, X, y, task_ids):
    ll = -mnl(beta, X, y, task_ids)
    lp = log_prior(beta)
    return ll + lp

n_steps = 11000
burn_in = 1000
np.random.seed(42)

beta_current = np.zeros(4)
samples = np.zeros((n_steps, 4))
log_post_current = log_posterior(beta_current, X, y, task_ids)

proposal_sd = np.array([0.05, 0.05, 0.05, 0.005])

for step in range(n_steps):
    beta_proposal = beta_current + np.random.normal(0, proposal_sd)
    log_post_proposal = log_posterior(beta_proposal, X, y, task_ids)
    
    log_ratio = log_post_proposal - log_post_current
    
    if np.log(np.random.uniform()) < log_ratio:
        beta_current = beta_proposal
        log_post_current = log_post_proposal
    
    samples[step] = beta_current

posterior_samples = samples[burn_in:]

posterior_means = np.mean(posterior_samples, axis=0)
posterior_std = np.std(posterior_samples, axis=0)
posterior_ci_lower = np.percentile(posterior_samples, 2.5, axis=0)
posterior_ci_upper = np.percentile(posterior_samples, 97.5, axis=0)

bayesian_results = pd.DataFrame({
    'Parameter': ['Netflix', 'Prime', 'Ads', 'Price'],
    'Posterior Mean': posterior_means,
    'Posterior Std': posterior_std,
    '95% CI Lower': posterior_ci_lower,
    '95% CI Upper': posterior_ci_upper
})

print("\nBayesian Estimation Results:")
print(bayesian_results.to_string(index=False))

Let's compare the results from both estimation methods and discuss their implications for understanding consumer preferences in the streaming service market.


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(posterior_samples[:,0])
plt.title("Trace plot: beta_Netflix")
plt.xlabel("Iteration")
plt.ylabel("Value")

plt.subplot(1,2,2)
plt.hist(posterior_samples[:,0], bins=30, density=True)
plt.title("Posterior: beta_Netflix")
plt.xlabel("Value")
plt.ylabel("Density")
plt.tight_layout()
plt.show()

The results from both estimation methods provide valuable insights into consumer preferences for streaming services. The parameter estimates indicate:


In [ ]:
import pandas as pd

summary_df = pd.DataFrame({
    'Parameter': ['Netflix', 'Prime', 'Ads', 'Price'],
    'MLE_Estimate': result.x,
    'MLE_StdErr': error,
    'MLE_95CI_Lower': ci_lower,
    'MLE_95CI_Upper': ci_upper,
    'Bayes_Mean': means,
    'Bayes_StdDev': stds,
    'Bayes_95CI_Lower': ci_lower,
    'Bayes_95CI_Upper': ci_upper
})

print(summary_df)

## 6. Discussion

_todo: Suppose you did not simulate the data. What do you observe about the parameter estimates? What does $\beta_\text{Netflix} > \beta_\text{Prime}$ mean? Does it make sense that $\beta_\text{price}$ is negative?_

To simulate data from a multi-level (random-parameter or hierarchical) model, we would allow each respondent to have their own set of preference parameters, rather than assuming everyone shares the same coefficients. This means, for each respondent, we would draw their individual-level betas from a population distribution (such as a multivariate normal centered at the overall mean). In estimation, we would use hierarchical Bayesian methods or mixed logit models to estimate both the distribution of preferences across the population and the individual-level parameters. This approach captures real-world heterogeneity in preferences and is more realistic for analyzing actual conjoint data.

_todo: At a high level, discuss what change you would need to make in order to simulate data from --- and estimate the parameters of --- a multi-level (aka random-parameter or hierarchical) model. This is the model we use to analyze "real world" conjoint data._

To move from a standard multinomial logit model to a multi-level (random-parameter or hierarchical) model, both the simulation and estimation processes need to be adapted to account for individual-level heterogeneity in preferences. In the simulation stage, rather than assigning the same set of part-worth utilities (betas) to every respondent, we would generate a unique set of coefficients for each individual. This is typically done by drawing each respondent’s betas from a population-level distribution, such as a multivariate normal distribution with a specified mean and covariance matrix. Each respondent’s choices would then be simulated using their own set of betas, allowing the simulated data to reflect realistic variation in preferences across individuals.
For estimation, we would use a hierarchical modeling approach, such as hierarchical Bayes or a mixed logit model. In these models, each respondent’s coefficients are treated as random effects, assumed to be drawn from a common population distribution whose parameters are also estimated from the data. The estimation process thus involves recovering both the distribution of preferences in the population and the individual-level coefficients. This is typically achieved using advanced computational methods, such as Markov Chain Monte Carlo (MCMC) for Bayesian estimation or simulated maximum likelihood for mixed logit models. By making these changes, the model can capture the true diversity of preferences found in real-world conjoint data, providing richer and more accurate insights into consumer choice behavior.
